In [19]:
import pandas as pd
import requests
import re
from collections import Counter
from typing import List, Dict

from consts import *

In [20]:
df = pd.read_csv('kns_csv_files/kns_committee.csv')
df = df[df['KnessetNum'] >= 25]
df = df[df['CategoryID'].isin([MONEY_COM_CATEGORY_ID, DEFENSE_COM_CATEGORY_ID, LAW_ORDER_COM_CATEGORY_ID, MESADERET_COM_CATEGORY_ID, KNESSET_COM_CATEGORY_ID])]
commitee_ids = df['CommitteeID'].to_list()

In [21]:
def get_meeting_protocol_text(text_path):
    # Define the URL to fetch
    base_url = 'https://production.oknesset.org/pipelines/data/committees/meeting_protocols_text/'
    # Send GET request to the URL
    response = requests.get(base_url + text_path)

    # Check if the request was successful (status code 200)
    if response.status_code == 200:
        response.encoding = 'utf-8'
        # Retrieve the content of the file
        return response.text
    else:
        raise ValueError(f"Failed to retrieve content. Status code: {response.status_code}")


In [22]:
com_session_df = pd.read_csv('kns_csv_files/kns_committeesession.csv')
com_session_df = com_session_df[com_session_df['CommitteeID'].isin(commitee_ids)]

com_session_df.dropna(subset=['text_parsed_filename'], inplace=True)
text_paths = com_session_df['text_parsed_filename'].to_list()
texts = [get_meeting_protocol_text(path) for path in text_paths]


In [23]:
knesset_members_df = pd.read_csv('kns_csv_files/kns_person.csv')
first_names, last_names = knesset_members_df['FirstName'].to_list(), knesset_members_df['LastName'].to_list()
knesset_members = [' '.join([first_name, last_name]) for first_name, last_name in zip(first_names, last_names)]

warnings = dict.fromkeys(knesset_members, [0, 0, 0])

# handle members with a middle name or a nickname
new_first_names, new_last_names = [], []

for fn, ln in zip(first_names, last_names):
    names = re.findall('\w+', fn)
    
    for name in names:
        warnings[name + ' ' + ln] = warnings[fn + ' ' + ln]
        new_first_names.append(name)
        new_last_names.append(ln)

# update first and last names
first_names += new_first_names
last_names += new_last_names

knesset_members = [' '.join([first_name, last_name]) for first_name, last_name in zip(first_names, last_names)]

In [33]:
def get_meeting_warnings(text, warnings, knesset_members) -> None:
    """
    Return warnings from the meeting protocol text.

    Parameters
    ----------
    text : str
        Meeting protocol text.

    warnings: Dict[str, List[int]]
        Number of warnings for each Knesset member.

    knesset_members: List[str]
        List of Knesset members.
    """
    regex = '<< דובר >>.+\n\n.+\n\n.+(?<=יו"ר).+\n\n.+(?=פעם|קריאה (ראש|שני|שליש))'

    # find all warnings
    matches = re.findall(regex, text)
    print(matches)
    for m in matches:
        sentences = m.split('\n')
        first_sentence, last_sentence = sentences[0], sentences[-1]
        for kns_member in knesset_members:
            if kns_member in first_sentence:
                word2idx = {'ראש': 0, 'שני': 1, 'שליש': 2}
                for word, idx in word2idx.items():
                    if word in last_sentence:
                        warnings[kns_member][idx] += 1
                        break
    
    

In [34]:
with open('2200827.txt', 'r', encoding='utf-8') as f:
    text = f.read()

get_meeting_warnings(text, warnings, knesset_members)

['ראש', 'ראש', 'ראש', '', '', '', '', '', '', 'שני', 'ראש', 'שני', 'שני', 'שני', 'שני', 'שני', 'ראש', 'שני', 'שני', '', '', '', '']


In [28]:
[w for w in warnings if warnings[w] != [0, 0, 0]]

[]